In [1]:
import pandas as pd
import numpy as np

In [39]:
# Load in the data

raw = pd.read_csv('../data/external/final_pphdatabase_merge_topost2.tab', sep='\t')

In [59]:
keep_cols = ['study', 'txtarm', 'pphloss', 'activebldcontrol', 'overallbld', 'age', 'iv_fluids', 'par_cat', 'q8_1', 'q9_1', 'q13_1', 'q10_2', 'q9_2', 'q8_2', 'q6_1', 'q7_1', 'q10_1', 'q11_1', 'q14_1', 'pphloss', 'q14_2']

In [60]:
raw['q14_2'].value_counts()

1    1337
2     424
3      26
Name: q14_2, dtype: int64

In [61]:
#need to transform q13_1 from weeks.days to days

df = raw[keep_cols].copy()
df['gestation_age'] = df['q13_1'].astype(str).str.split('.', expand=True)[0].astype(float) * 7 + df['q13_1'].astype(str).str.split('.', expand=True)[1].astype(float)
df.drop(columns=['q13_1'], inplace=True)

In [63]:
df.to_csv('../data/processed/goodvars.csv', index=False)